In [ ]:
import tensorflow as tf
import time
import numpy as np
from tensorflow.keras.datasets import cifar10
import os
import subprocess

In [ ]:
(x_test, y_test), _ = cifar10.load_data()
x_test = x_test.astype('float32') / 255.0
y_test = tf.keras.utils.to_categorical(y_test, 10)

In [ ]:
def benchmark_model(model_path, x_test, y_test, output_name, batch_size=32):
    # Load model
    model = tf.saved_model.load(model_path)
    infer = model.signatures["serving_default"]

    # Benchmark latency and throughput
    num_batches = len(x_test) // batch_size
    latencies = []
    total_samples = 0
    start_total_time = time.time()

    for i in range(num_batches):
        batch_x = x_test[i * batch_size : (i + 1) * batch_size]
        start_time = time.time()
        _ = infer(tf.constant(batch_x))
        latencies.append(time.time() - start_time)
        total_samples += len(batch_x)

    end_total_time = time.time()
    avg_latency = np.mean(latencies)
    throughput = total_samples / (end_total_time - start_total_time)

    # Evaluate model accuracy
    predictions = []
    for i in range(num_batches):
        batch_x = x_test[i * batch_size : (i + 1) * batch_size]
        outputs = infer(tf.constant(batch_x))
        predictions.extend(np.argmax(outputs[output_name].numpy(), axis=1))

    accuracy = np.mean(np.argmax(y_test[:num_batches * batch_size], axis=1) == predictions)
    return avg_latency, throughput, accuracy

In [ ]:
def get_model_size(model_path):
    total_size = 0
    for dirpath, dirnames, filenames in os.walk(model_path):
        for f in filenames:
            fp = os.path.join(dirpath, f)
            total_size += os.path.getsize(fp)
    return total_size / (1024 * 1024)  # Convert bytes to MB

In [ ]:
def get_gpu_memory():
    try:
        result = subprocess.run(
            ["nvidia-smi", "--query-gpu=memory.used", "--format=csv,nounits,noheader"],
            stdout=subprocess.PIPE,
            text=True
        )
        memory_used = int(result.stdout.splitlines()[0])  # Assume single GPU
        return memory_used
    except FileNotFoundError:
        print("nvidia-smi not found. Skipping GPU memory measurement.")
        return None

In [ ]:
keras_model_path = "/content/drive/MyDrive/data_aug_model"
tensorrt_model_path = "/content/drive/MyDrive/optimized_model"
keras_output = 'dense_3'
tensor_rt_output = 'output_0'

In [ ]:
keras_latency, keras_throughput, keras_accuracy = benchmark_model(keras_model_path, x_test, y_test, keras_output)
keras_size = get_model_size(keras_model_path)
keras_memory = get_gpu_memory()
print(f"Keras Model - Latency: {keras_latency:.4f}s, Throughput: {keras_throughput:.2f} samples/s, "
      f"Accuracy: {keras_accuracy:.4%}, Size: {keras_size:.2f} MB, GPU Memory: {keras_memory} MB")

Keras Model - Latency: 0.0081s, Throughput: 3927.45 samples/s, Accuracy: 91.3852%, Size: 32.57 MB, GPU Memory: 635 MB
